In [1]:
import logging

from time import time, sleep
from multiprocessing import Process, Manager
logging.basicConfig()

from kazoo.client import KazooClient

In [2]:
class Philosopher(Process):
    def __init__(self, root: str, id: int, fork_path: str, eat_seconds: int = 15, max_id: int = 5):
        super().__init__()
        self.url = f'{root}/{id}'
        self.root = root
        self.fork = fork_path
        self.id = id
        self.left_fork_id = id
        self.right_fork_id = id + 1 if id + 1 < max_id else 0
        self.eat_seconds = eat_seconds
        self.partner_id_left = id - 1 if id - 1 >= 0 else max_id-1
        self.partner_id_right = id + 1 if id + 1 < max_id else 0
        
    def run(self):
        zk = KazooClient()
        zk.start()
        # Для каждого философа определяем путь до стола и его вилок
        table_lock = zk.Lock(f'{self.root}/table', self.id)
        left_fork = zk.Lock(f'{self.root}/{self.fork}/{self.left_fork_id}', self.id)
        right_fork = zk.Lock(f'{self.root}/{self.fork}/{self.right_fork_id}', self.id)

        start = time()
        while time() - start < self.eat_seconds:
            
            # Делаем так, чтобы только 1 филосов мог проверять вилки
            with table_lock:
                # Свободны ли вилки? Также добавляем проверку на то, чтобы философы ели примерно одинаково
                if len(left_fork.contenders()) == 0 and len(right_fork.contenders()) == 0 \
                        and counters[self.partner_id_right] >= counters[self.id] \
                        and counters[self.partner_id_left] >= counters[self.id]:
                    # Берём вилки
                    left_fork.acquire()
                    right_fork.acquire()
            # Вилки подняты, начинаем есть
            if left_fork.is_acquired and right_fork.is_acquired:
                print(f'Philosopher {self.id}: Im eating')
                counters[self.id] += 1
                # Пережёвывать еду очень важный и долгий процесс
                sleep(WAIT_EAT_MS)
                # Поел? положи вилки!!!
                left_fork.release()
                right_fork.release() 
            else:
                print(f'Philosopher {self.id}: Im thinking') # Нет вилок, так думай как их достать
            sleep(WAIT_AFTER_ALL_DONE)
            
        print(f'Philosopher with id={self.id} eat counter={counters[self.id]}')
        zk.stop()
        zk.close()

In [3]:
root = '/task1'
fork_path = 'forks'
seconds_eat = 30
WAIT_EAT_MS = 1.25
WAIT_AFTER_ALL_DONE = 0.4

master_zk = KazooClient()
master_zk.start()

if master_zk.exists(root):
    master_zk.delete(root, recursive=True)

master_zk.create(root)
master_zk.create('/task1/table')
master_zk.create('/task1/forks')
for i in range(0,5):
    master_zk.create(f'/task1/forks/{i}')

counters = Manager().list()
p_list = list()
for i in range(0, 5):
    p = Philosopher(root, i, fork_path, seconds_eat)
    counters.append(0)
    p_list.append(p)
    
for p in p_list: 
    p.start()

Philosopher 0: Im eating
Philosopher 1: Im thinking
Philosopher 2: Im eating
Philosopher 3: Im thinking
Philosopher 4: Im thinking
Philosopher 1: Im thinking
Philosopher 3: Im thinking
Philosopher 4: Im thinking
Philosopher 1: Im thinking
Philosopher 3: Im thinking
Philosopher 4: Im thinking
Philosopher 1: Im thinking
Philosopher 3: Im thinking
Philosopher 4: Im thinking
Philosopher 1: Im eating
Philosopher 3: Im eating
Philosopher 0: Im thinking
Philosopher 4: Im thinking
Philosopher 2: Im thinking
Philosopher 0: Im thinking
Philosopher 4: Im thinking
Philosopher 2: Im thinking
Philosopher 0: Im thinking
Philosopher 4: Im thinking
Philosopher 2: Im thinking
Philosopher 0: Im thinking
Philosopher 4: Im thinking
Philosopher 2: Im eating
Philosopher 1: Im thinking
Philosopher 0: Im thinking
Philosopher 3: Im thinking
Philosopher 4: Im eating
Philosopher 1: Im thinking
Philosopher 0: Im thinking
Philosopher 3: Im thinking
Philosopher 1: Im thinking
Philosopher 0: Im thinking
Philosopher 3